In [27]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from math import sqrt, ceil, floor, log, pi

from tensorflow.keras import layers, losses, models as mds, optimizers

In [39]:
# Dataset image size
IMG_SIZE = 264
N_CLASSES = 102

def preprocess(image, *args):
    image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
    image /= 255
    return (image, *args)

train_ds, val_ds = tfds.load(
    'oxford_flowers102',
    split=['train[:20%]', 'validation'],
    as_supervised=True,
    read_config=tfds.ReadConfig(try_autocache=False)
)

train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [47]:
LEARNING_RATE = 0.1

def conv_block(x, filters, strides, padding, activation):
    kwargs = dict(strides=strides, padding=padding)
    layer = tf.keras.Sequential([
        layers.Conv2D(filters, 3, **kwargs),
        layers.BatchNormalization(),
        layers.Activation(activation),
#         layers.Dropout(.65)
    ])
    
    return layer(x)

def sic_block(input_tensor, filters, strides, padding, activation):
    def calculate_variants(size):
        return ceil(log(ceil(sqrt(size)), pi))
    
    def variant_dims(count):
        return list(map(lambda x: 3**x, range(1, count+1)))
    
    def make_layer(size):
        kwargs = dict(strides=strides, padding=padding)
        return layers.Conv2D(filters, size, **kwargs)

    size = min(input_tensor.shape[1:-1])
    variants = variant_dims(calculate_variants(size))
    conv_layers = map(make_layer, variants)
    conv_outputs = list(map(lambda x: x(input_tensor), conv_layers))
    merged = tf.concat(conv_outputs, axis=-1)
    normalized = layers.BatchNormalization()(merged)
    output = layers.Activation(activation)(normalized)
#     reg_output = layers.Dropout(.65)(output)

    return output

def sidsc_block(input_tensor, filters, strides, padding, activation):
    def calculate_variants(size):
        return ceil(log(ceil(sqrt(size)), pi))
    
    def variant_dims(count):
        return list(map(lambda x: 3**x, range(1, count+1)))
    
    def make_layer(size):
        kwargs = dict(strides=strides, padding=padding)
        return layers.DepthwiseConv2D(size, **kwargs)

    size = min(input_tensor.shape[1:-1])
    variants = variant_dims(calculate_variants(size))
    sidsc_layers = map(make_layer, variants)
    
    x = list(map(lambda x: x(input_tensor), sidsc_layers))
    x = tf.math.add_n(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    
    # Channel Expansion
    kwargs = dict(strides=1, padding=padding, activation=activation)
    output = layers.Conv2D(filters, variants[0], **kwargs)(x)
#     reg_output = layers.Dropout(.65)(output)

    return output

def sic_specs():
    return [(8, 2), (16, 1), (32, 2), (32, 1), (64, 2), (64, 1), (128, 2), (256, 2)]
    
def conv_specs():
    return [(32, 2), (64, 1), (128, 2), (128, 1), (256, 2), (256, 1),
        (512, 2), (512, 1), (512, 1), (512, 1), (512, 1), (512, 1), (1024, 2)]

def sidsc_specs():
    return [(32, 2), (64, 1), (128, 2), (128, 1), (256, 2), (256, 1), (512, 2), (512, 1), (1024, 2)]

def create_model(conv_type=None):
    def conv_fn(x, y):
        args = [x, y[0], y[1], 'same', 'relu']

        if conv_type == 'sic':
            return sic_block(*args)
        elif conv_type == 'sidsc':
            return sidsc_block(*args)
        else:
            return conv_block(*args)
    
    def spec_fn():
        if conv_type == 'sic':
            return sic_specs()
        elif conv_type == 'sidsc':
            return sidsc_specs()
        else:
            return conv_specs()
        
    inp = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    specs = spec_fn()

    x = reduce(conv_fn, specs, inp)
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(N_CLASSES, activation='softmax')(x)

    model = tf.keras.Model(inputs=inp, outputs=output)
#     optimizer = optimizers.Nadam(learning_rate=LEARNING_RATE)
    optimizer = optimizers.Adadelta(learning_rate=LEARNING_RATE)
    loss = 'sparse_categorical_crossentropy'

    model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
    
    return model

model = create_model(conv_type='sidsc')
# model = create_model(conv_type='sic')
# model = create_model()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 264, 264, 3  0           []                               
                                )]                                                                
                                                                                                  
 depthwise_conv2d_62 (Depthwise  (None, 132, 132, 3)  30         ['input_9[0][0]']                
 Conv2D)                                                                                          
                                                                                                  
 depthwise_conv2d_63 (Depthwise  (None, 132, 132, 3)  246        ['input_9[0][0]']                
 Conv2D)                                                                                    